In [41]:
from camelup.utilities import *
from camelup.treesearch import * 
from camelup import camelup, config
from copy import deepcopy

In [42]:
game = camelup.Game(3)

camel_dict = {
    "red": {"height": 1, "space": 1, "need_roll": True},
    "blue": {"height": 2, "space": 1, "need_roll": True},
    "green": {"height": 1, "space": 2, "need_roll": True},
}
game.camel_dict = camel_dict

game.play_winner_card("blue")
game.play_loser_card("red")
game.play_bet_tile("blue")

game.state = 2
game.play_bet_tile("blue")
game.play_winner_card("green")

In [43]:
def parse_dump(string):
    """Gets information from the AST dump and return the AST object and the id

    Parameters
    ----------
    string : str
        Tree to extract the id for

    Returns
    -------
    str
        A string with the object and name info

    """
    re1 = "id='(.*?)',"
    return string.split('(')[0] + ' - ' + re.search(re1, string).group(1)


def create_benchmark_func(tree, performance):
    """Creating a benchmark function named benchmark_`func`

    Parameters
    ----------
    tree : str
        AST for function to create benchmark function for
    performance : dict
        Dictionary to capture function

    Returns
    -------
    str :
        Tree of the benchmark function

    """
    t = 0
    new_first_layer = list()
    for node in tree.body[0].body:
        if node.__class__ != ast.Return:
            node_dump = ast.dump(node)
            parsed_dump = parse_dump(node_dump)
            performance[parsed_dump] = list()
            start_time = ast.parse(f"t{t} = time()").body[0]
            t+=1
            end_time = ast.parse(f"t{t} = time()").body[0]
            dif = ast.parse(f"performance['{parsed_dump}'].append(t{t}-t{t-1})").body[0]
            new_first_layer.extend([start_time, node, end_time, dif])
            t+=1
        else:
            new_first_layer.extend([node])
    tree.body[0].body = new_first_layer
    tree.body[0].name = f'benchmark_{tree.body[0].name}'
    return tree


In [44]:
func_str = inspect.getsource(calc_utility)
tree = ast.parse(StringIO(func_str).read())
global performance
performance = dict()
exec(compile(create_benchmark_func(tree, performance), filename="<ast>", mode="exec"))
print(ast.dump(tree))

IndentationError: unexpected indent (<unknown>, line 1)

In [33]:
for i in range(100):
    benchmark_calc_utility(game,5)

In [34]:
performance

{'Assign - coins': [0.03619098663330078,
  2.7894973754882812e-05,
  2.288818359375e-05,
  1.430511474609375e-05,
  1.5020370483398438e-05,
  1.4066696166992188e-05,
  1.52587890625e-05,
  1.4781951904296875e-05,
  1.4066696166992188e-05,
  1.5020370483398438e-05,
  1.4781951904296875e-05,
  1.5020370483398438e-05,
  1.4066696166992188e-05,
  1.7881393432617188e-05,
  1.3828277587890625e-05,
  1.5020370483398438e-05,
  1.52587890625e-05,
  1.5020370483398438e-05,
  1.4781951904296875e-05,
  1.4781951904296875e-05,
  1.621246337890625e-05,
  1.5020370483398438e-05,
  1.5020370483398438e-05,
  1.4781951904296875e-05,
  1.4066696166992188e-05,
  1.3828277587890625e-05,
  1.71661376953125e-05,
  2.5987625122070312e-05,
  1.4066696166992188e-05,
  1.6927719116210938e-05,
  1.5020370483398438e-05,
  1.3828277587890625e-05,
  1.3828277587890625e-05,
  1.5020370483398438e-05,
  1.3828277587890625e-05,
  2.47955322265625e-05,
  1.5020370483398438e-05,
  1.52587890625e-05,
  1.5974044799804688e-

In [6]:
game

In [22]:
util.benchmark_calc_utility

AttributeError: module 'camelup.utilities' has no attribute 'benchmark_calc_utility'

In [8]:
benchmark_calc_utility(game, 5)

NameError: name 'benchmark_calc_utility' is not defined

In [9]:
game